In [1]:
import os
import pickle as pk
import tensorflow as tf
from __future__ import absolute_import, division, print_function
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import unicodedata
import re
import os
import time

import json
from glob import glob
from PIL import Image
import pickle
import pandas as pd
ACCEPTABLE_AVAILABLE_MEMORY = 8000
import subprocess as sp

os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
def mask_unused_gpus(leave_unmasked=1,random=True):
    COMMAND = "nvidia-smi --query-gpu=memory.free --format=csv"
    try:
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        memory_free_info = _output_to_list(sp.check_output(COMMAND.split()))[1:]
        memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
        available_gpus = [i for i, x in enumerate(memory_free_values) if x > ACCEPTABLE_AVAILABLE_MEMORY]
        import numpy as np
        #print(available_gpus)
        if random:
           available_gpus = np.asarray(available_gpus)
           np.random.shuffle(available_gpus)
        if len(available_gpus) < leave_unmasked: raise ValueError('Found only %d usable GPUs in the system' %\
                                                                  len(available_gpus))
        os.environ["CUDA_VISIBLE_DEVICES"] = ','.join(map(str, available_gpus[:leave_unmasked]))
        #print(','.join(map(str, available_gpus[:leave_unmasked])))
    except Exception as e:
        print('"nvidia-smi" is probably not installed. GPUs are not masked', e)

#mask_unused_gpus(2)
mask_unused_gpus(1)


import os
import re
import time
import datetime
import sys
sys.path.append('../src')
#sys.path.append('/content/drive/MyDrive/PerformanceNarrativesRuns/')

import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')
nltk.download('brown')
import functools

from nltk import wordpunct_tokenize,word_tokenize
# Read the dataset
full_data= json.load(open('../dataset/annotation_data_with_augmentations.json'))['data']
#jdata = json.load(open('james_submission.json'))
first_batch= json.load(open('../dataset/annotation_download.json'))
full_data = train_data = full_data#+first_batch
test_data = json.load(open('../dataset/test set.json'))
classes_tokens =[f'C{i}' for i in range(1,6)]

[nltk_data] Downloading package punkt to /home/essel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /home/essel/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
from data_utils import *

In [3]:
pc= full_data[123]
examples = composePreambleAndInputs(pc,identical_metrics=identicals,augnment_metrics=True,augment_output_order=True)
examples

{'preamble': '<MetricsInfo> precision | VALUE_MODERATE | 74.65% <|> recall | VALUE_MODERATE | 77.56% <|> accuracy | VALUE_MODERATE | 75.49%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|BALANCED|>'],
 'metrics': ['recall', 'accuracy', 'precision'],
 'values': ['77.56%', '75.49%', '74.65%'],
 'rates': ['MODERATE', 'MODERATE', 'MODERATE'],
 'narration': 'The classifier trained to solve the given classification problem was evaluated based on its scores across the following metrics: accuracy, recall and precision. For the accuracy, the model achieved 75.49%, and 77.56% for the recall with a moderate precision score of (74.65%). Considering these values, we can make the conclusion that this model can correctly differentiating between the examples belonging to the different class labels with a close to moderate chance of misclassification.'

In [4]:
processInputTableAndNarrations = composePreambleAndInputs

In [5]:
test_sample = []
eval_tables = []
for pc in test_data:
    test_sample.append(processInputTableAndNarrations(pc,identical_metrics=identicals))
    #eval_tables.append(parseTableStructureForEval(pc,identicals))
rtest_sample = []
reval_tables = []
for pc in test_data:
    rtest_sample.append(processInputTableAndNarrations(pc,identical_metrics=identicals))
    #reval_tables.append(parseTableStructureForEval(pc,identicals))

In [7]:
test_sample[0]

{'preamble': '<MetricsInfo> precision | VALUE_HIGH | 91.3% <|> sensitivity | VALUE_HIGH | 87.29% && sensitivity | also_known_as | recall <|> accuracy | VALUE_HIGH | 90.67% <|> f1score | VALUE_HIGH | 88.89%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|IMBALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|IMBALANCED|>'],
 'metrics': ['sensitivity', 'accuracy', 'precision', 'f1score'],
 'values': ['87.29%', '90.67%', '91.3%', '88.89%'],
 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH'],
 'narration': "The classifier was able to produce fairly high scores across the metrics sensitivity, accuracy, precision and F1score. Specifically, for the sensitivity it scored 87.29%, accuracy (96.67%) and precision (91.3%) with the F1score equal to 88.89%. These scores suggest that the model will incorrectly assign the wrong labels for only a small number of test cases. Overall, the model's prediction decisio

In [7]:
mask_met=False
processed=[]
preambles = []
# apply the preprocessing to the data
for idx,pc in enumerate(full_data):
    #print(idx)
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=False,)
    preambles.append(examples['preamble'])
    processed.append(examples)

for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if len(examples['classes'])> 2:
        if examples['preamble'] not in set(preambles):
            preambles.append(examples['preamble'])
            processed.append(examples)
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)        
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if examples['preamble'] not in set(preambles):
        preambles.append(examples['preamble'])
        processed.append(examples)


sample_rounds = 10
sample_size = 500
for i in range(sample_rounds):
    for idx,pc in enumerate(random.sample(full_data,sample_size)):
        examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
        if examples['preamble'] not in set(preambles):
            preambles.append(examples['preamble'])
            processed.append(examples)        
        
random.shuffle(processed)
random.shuffle(processed)

In [8]:
len(preambles)

4977

In [9]:
len(preambles)

4977

In [3]:
def cleanRatingPreamble(preamble):
    # This function will replace the VALUE_LOW, VALUE_HIGH and VALUE_MODERATE mentions in the preamble a common token VALUE
    preamble_dict = {'VALUE_LOW':'VALUE','VALUE_HIGH':'VALUE','VALUE_MODERATE':'VALUE'}
    preamble = [functools.reduce(lambda a, kv: a.replace(*kv),
                preamble_dict.items(),
                                 re.sub('\s+', ' ', ss.strip().replace('\n', ' '))) for ss in [preamble]][0]
    return preamble

In [16]:
cleanRatingPreamble(preambles[0])

'<MetricsInfo> accuracy | VALUE | 89.12% <|> auc | VALUE | 96.08% <|> recall | VALUE | 94.72% <|> precision | VALUE | 82.64% <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA and #CB <|section-sep|> <|table2text|>'

In [10]:
processed[12]

{'preamble': '<MetricsInfo> sensitivity | VALUE_HIGH | 88.89% && sensitivity | also_known_as | recall <|> accuracy | VALUE_HIGH | 85.53% <|> f2score | VALUE_HIGH | 88.15% <|> precision | VALUE_HIGH | 85.33% <|> specificity | VALUE_HIGH | 82.59%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA and #CB  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB'],
 'dataset_attribute': ['<|BALANCED|>'],
 'metrics': ['specificity', 'accuracy', 'sensitivity', 'precision', 'f2score'],
 'values': ['82.59%', '85.53%', '88.89%', '85.33%', '88.15%'],
 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH', 'HIGH'],
 'narration': "Evaluating the classifier's prowess on the classification task produced the scores 85.33%, 88.89%, 82.59%, 85.53%, and 88.15%, respectively, across the metrics precision, sensitivity, specificity, accuracy, and F2score. The difference between the precision, and sensitivity scores indicates that the classifier is very confident ab

In [10]:
for idx,pc in enumerate(full_data):
    examples = processInputTableAndNarrations(pc,
                                              identical_metrics=identicals,
                                              augnment_metrics=True,
                                              augment_output_order=True)
    if len(examples['classes'])> 2:
        if examples['preamble'] not in set(preambles):
            preambles.append(examples['preamble'])
            processed.append(examples)

random.shuffle(processed)
random.shuffle(processed)

In [13]:
len(processed)

2806

In [4]:
import pickle as pk

In [5]:
# Clean all the preamble to strip them of the ratings
for pc in processed:
    pc['raw_preamble'] = cleanRatingPreamble(pc['preamble'])

NameError: name 'processed' is not defined

In [6]:
# Save the dataset
#pk.dump(processed,open('../dataset/train_dataset_full.dat','wb'))
processed = pk.load(open('../dataset/train_dataset_full.dat','rb'))

In [7]:
len(processed)

4977

In [8]:
processed[593]

{'preamble': '<MetricsInfo> precision | VALUE_HIGH | 90.98% <|> accuracy | VALUE_HIGH | 93.42% <|> recall | VALUE_HIGH | 88.13% <|> f1score | VALUE_HIGH | 89.42%  <|section-sep|> <TaskDec> ml_task | dataset_attributes | <|BALANCED|> && ml_task | class_labels | #CA, #CB and #CC  <|section-sep|> <|table2text|> ',
 'classes': ['#CA', '#CB', '#CC'],
 'dataset_attribute': ['<|BALANCED|>'],
 'metrics': ['precision', 'recall', 'f1score', 'accuracy'],
 'values': ['90.98%', '88.13%', '89.42%', '93.42%'],
 'rates': ['HIGH', 'HIGH', 'HIGH', 'HIGH'],
 'narration': "The algorithm's prediction performance on the given multi-class classification problem where the test instances are classified as either #CA or #CB or #CC is: (a) Accuracy = 93.42%. (b) Precision = 90.98%. (c) F1score = 89.42%. (d) Recall = 88.13%. On this multi-class problem, the algorithm is shown to perform very well across all the evaluation metrics under consideration. The scores across the different metrics indicate that it is ver

In [9]:
import pickle as pk
from model_utils import setupTokenizer
modeltype = 'baseline'
modelbase='facebook/bart-large'
tokenizer = tokenizer_ = setupTokenizer(modelbase=modelbase)

In [12]:
dataset = RDFDataSetForTableStructured(tokenizer_,  processed, modelbase,max_preamble_len=160,
                                       max_len_trg=180, max_rate_toks=8,
                                       lower_narrations=True, process_target=True,use_raw=False)

In [15]:
dataset.processTableInfo(processed[593])

{'preamble_tokens': tensor([    0, 50267,  5234, 37938,  1721, 50270, 15483,  1814,     4,  5208,
           207, 50265,  7904, 45386,  1721, 50270, 15483,  8060,     4,  3714,
           207, 50265, 13139,  1250,  1721, 50270, 15483,  7953,     4,  1558,
           207, 50265,   506,   134, 31673,  1721, 50270, 15483,  8572,     4,
          3714,   207, 50269, 50266, 50281, 15483, 50287, 15483, 50283, 49145,
         50281, 15483, 50284, 15483, 50273,     6, 50274,   463, 50275, 50269,
         50268,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,    

In [14]:
dataset[0]

{'preamble_tokens': tensor([    0, 50267,  5234, 37938,  1721, 50271, 15483,  5545,     4,  6668,
           207, 50265, 13139,  1250,  1721, 50271, 15483,  3620,     4,  4563,
           207, 50265,   506,   176, 31673,  1721, 50271, 15483,  4268,     4,
          2546,   207, 50265,  7904, 45386,  1721, 50272, 15483,  2491,     4,
          3272,   207, 50269, 50266, 50281, 15483, 50287, 15483, 50283, 49145,
         50281, 15483, 50284, 15483, 50273,     6, 50274,   463, 50275, 50269,
         50268,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,    